Notes on MLFlow:
- 'Run name' field: model name, type of output (multilabel vs unilabel), which vaccine (for multiclass only)
- 'Parameters' field: methods applied for data (data cleaning, data balancing, hyperparameters)--insert feature engineering info here, if relevant?
- 'Tags' field: details about the features used for the run (is one of the vaccines in the features?)

In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings

#mlflow import
import mlflow
from modeling.config import EXPERIMENT_NAME_multilabel, EXPERIMENT_NAME_h1n1, EXPERIMENT_NAME_seasonal, EXPERIMENT_NAME_multiclass
TRACKING_URI = open("../.mlflow_uri").read().strip()

RSEED = 42

# Modeling Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

!pip install plotly
import plotly.express as px


from sklearn.dummy import DummyClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm


warnings.filterwarnings('ignore')


You should consider upgrading via the '/Users/julianeberek/neuefische/TheFluShot/.venv/bin/python -m pip install --upgrade pip' command.


Import data from a previously prepared dataframe:

In [27]:
df = pd.read_csv('../data/Flu_Shot_Data_cleaned_2.csv')

In [28]:
df.head()

,Unnamed: 0,h1n1_vaccine,seasonal_vaccine,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,0,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,0,0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,0,1,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,0,0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


Dropping the strange 'Unnamed: 0' column:

In [29]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [30]:
df.shape

(26707, 37)

# Set up for modelling (stays the same for all experiemnts)

Set up of pipeline preprocessor:

In [31]:
# Pipeline for categorical features
# This stays the same for everything
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown='error', drop='first'))
])

In [32]:
cat_features = list(df.columns)

Instantiating the Random Forest Model 

In [33]:
# for Random Forest
rand_forst= RandomForestClassifier()

# TheFluShot_multilabel: Multilabel prediction (both vaccinations)

Removal of target variables from cat_features list (this needs to be adjusted for each dataset):

In [34]:
cat_features_no_vacc = cat_features.copy()

In [35]:
cat_features_no_vacc.remove('h1n1_vaccine')

In [36]:
cat_features_no_vacc.remove('seasonal_vaccine')

In [37]:
cat_features_no_vacc

['h1n1_concern',
 'h1n1_knowledge',
 'behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face',
 'doctor_recc_h1n1',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'child_under_6_months',
 'health_worker',
 'health_insurance',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'age_group',
 'education',
 'race',
 'sex',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'employment_status',
 'hhs_geo_region',
 'census_msa',
 'household_adults',
 'household_children',
 'employment_industry',
 'employment_occupation']

Rename the features and target to 'X' and 'y', to make the test-train split easier:

In [38]:
y_both_vacc = df[['h1n1_vaccine', 'seasonal_vaccine']].copy()

In [39]:
y_both_vacc = y_both_vacc.to_numpy()
y_both_vacc

array([[0, 0],
       [0, 1],
       [0, 0],
       ...,
       [0, 1],
       [0, 0],
       [0, 0]])

In [40]:
#NB: dropping the 'h1n1_vaccine' and 'seasonal_vaccine' columns
X_no_vacc = df.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])

Performing test-train split (the same data can be used for each model in multilabelling):

In [41]:
X_no_vacc_train, X_no_vacc_test, y_both_vacc_train, y_both_vacc_test = train_test_split(X_no_vacc, y_both_vacc, stratify = y_both_vacc, test_size=0.2, random_state=RSEED)

In [42]:
print('X_no_vacc_train shape:', X_no_vacc_train.shape)
print('X_no_vacc_test shape:', X_no_vacc_test.shape)
print('y_both_vacc_train:', y_both_vacc_train.shape)
print('y_both_vacc_test:', y_both_vacc_test.shape)

X_no_vacc_train shape: (21365, 35)
X_no_vacc_test shape: (5342, 35)
y_both_vacc_train: (21365, 2)
y_both_vacc_test: (5342, 2)


Setting up the preprocessor (the same one can be used for each modelling in multilabelling):

In [43]:
preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cat_features_no_vacc)
])

Setting up the multilabel estimators for Random Forest

In [44]:
# for Random Forest
multilabel_est_rand_forst= MultiOutputClassifier(
    estimator=rand_forst
)

Setting up the pipeline for Random Forest

In [45]:
# for Random Forest

rand_forst_multilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", multilabel_est_rand_forst),
])

### Hyperparameter Tuning for the Random Forest model

In [21]:
multilabel_est_rand_forst.get_params().keys()

dict_keys(['estimator__bootstrap', 'estimator__ccp_alpha', 'estimator__class_weight', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__max_samples', 'estimator__min_impurity_decrease', 'estimator__min_impurity_split', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__oob_score', 'estimator__random_state', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'n_jobs'])

In [22]:
# Hyperparameter grid from a previous notebook 
# will need to be adjusted in further runs 
# the pipeline step is accessed with 'estimators__estimator__'

param_grid_multilabel = {
    'estimators__estimator__n_estimators': np.linspace(10, 100, 200).astype(int),
    'estimators__estimator__max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'estimators__estimator__max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'estimators__estimator__max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    #'estimators__estimator__min_samples_split': [2, 5, 10],
    'estimators__estimator__bootstrap': [True, False]
}

In [24]:
# the random search model is created 
# we define roc_auc as our evaluation metric for the parameter search 
# set verbose to a higher number next time 

grid_rand_forst_multilabel = RandomizedSearchCV(rand_forst_multilabel_pipeline, param_distributions=param_grid_multilabel, cv=5, scoring='roc_auc', 
                           verbose=5, n_jobs=-1, n_iter=100)

Training the Random Forest grid search model 

In [ ]:
# for Random Forest
rand_forst_multilabel = grid_rand_forst_multilabel.fit(X_no_vacc_train,  y_both_vacc_train)

In [26]:
# Show best parameters
print('Best score:\n{:.2f}'.format(rand_forst_multilabel.best_score_))
print("Best parameters:\n{}".format(rand_forst_multilabel.best_params_))

Best score:
0.84
Best parameters:
{'estimators__estimator__n_estimators': 94, 'estimators__estimator__max_leaf_nodes': 45, 'estimators__estimator__max_features': 0.5, 'estimators__estimator__max_depth': 11, 'estimators__estimator__bootstrap': True}


In [30]:
# Save best model

best_model_rand_forst_multi = rand_forst_multilabel.best_estimator_['estimators']

In [31]:
best_model_rand_forst_multi

MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=11,
                                                       max_features=0.5,
                                                       max_leaf_nodes=45,
                                                       n_estimators=94))

In [33]:
# Creating an adapted pipeline 

rand_forst_multilabel_pipeline_tuned = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", best_model_rand_forst_multi),
])

In [34]:
# fitting the data to the new tuned model 

best_model_rand_forst_multi = rand_forst_multilabel_pipeline_tuned.fit(X_no_vacc_train,  y_both_vacc_train)

In [ ]:
# Figure out later what this does and if we want to use it


#y_train_predicted = cross_val_predict(full_pipeline, X_train, y_train, cv=5)

Making predictions based on train and test data:

In [35]:
# for Random Forest
rand_forst_multilabel_trainpreds_best = best_model_rand_forst_multi.predict(X_no_vacc_train)
rand_forst_multilabel_testpreds_best = best_model_rand_forst_multi.predict(X_no_vacc_test)

# for SVM
#svm_multilabel_trainpreds = svm_multilabel.predict(X_no_vacc_train)
#svm_multilabel_testpreds = svm_multilabel.predict(X_no_vacc_test)

### Model evaluation

#### Train data

In [36]:
# Random Forest--Train data evaluation Metrics for H1N1 Vaccines
# THESE RESULTS ARE SLIGHTLY OFF FROM PREVIOUS NOTEBOOK (BY 0.01)--CHECK WHAT'S AT PLAY. VARIATION COMING FROM TEST-TRAIN SPLIT? EXPORT TEST-TRAIN DATA AS CSV AND TRY AGAIN
#IS IT BECAUSE IT'S RANDOM FOREST? DIFFERENT STARTING STUMP EACH TIME?
h1n1_rand_forst_multilabel_train_acc_best = accuracy_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds_best[:, 0])
h1n1_rand_forst_multilabel_train_recall_best = recall_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds_best[:, 0])
h1n1_rand_forst_multilabel_train_precision_best = precision_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds_best[:, 0])
h1n1_rand_forst_multilabel_train_f1_best = f1_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds_best[:, 0])
h1n1_rand_forst_multilabel_train_roc_best = roc_auc_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds_best[:, 0])

#print("Accuracy: {:.2f}".format(accuracy_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds[:, 0])))
#print("Recall: {:.2f}".format(recall_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds[:, 0])))
#print("Precision: {:.2f}".format(precision_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds[:, 0])))
#print("F1: {:.2f}".format(f1_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds[:, 0])))
#print("ROC: {:.2f}".format(roc_auc_score(y_both_vacc_train[:, 0], rand_forst_multilabel_trainpreds[:, 0])))

In [37]:
# Random Forest--Train data Evaluation Metrics for seasonal Vaccines
# THESE RESULTS ARE SLIGHTLY OFF FROM PREVIOUS NOTEBOOK (BY 0.01)--CHECK WHAT'S AT PLAY. VARIATION COMING FROM TEST-TRAIN SPLIT? EXPORT TEST-TRAIN DATA AS CSV AND TRY AGAIN
#IS IT BECAUSE IT'S RANDOM FOREST? DIFFERENT STARTING STUMP EACH TIME?
seasonal_rand_forst_multilabel_train_acc_best = accuracy_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds_best[:, 1])
seasonal_rand_forst_multilabel_train_recall_best = recall_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds_best[:, 1])
seasonal_rand_forst_multilabel_train_precision_best = precision_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds_best[:, 1])
seasonal_rand_forst_multilabel_train_f1_best = f1_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds_best[:, 1])
seasonal_rand_forst_multilabel_train_roc_best = roc_auc_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds_best[:, 1])

#print("Accuracy: {:.2f}".format(accuracy_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds[:, 1])))
#print("Recall: {:.2f}".format(recall_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds[:, 1])))
#print("Precision: {:.2f}".format(precision_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds[:, 1])))
#print("F1: {:.2f}".format(f1_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds[:, 1])))
#print("ROC: {:.2f}".format(roc_auc_score(y_both_vacc_train[:, 1], rand_forst_multilabel_trainpreds[:, 1])))

In [ ]:
# SVM--Train data evaluation Metrics for H1N1 Vaccines
#h1n1_svm_multilabel_train_acc = accuracy_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])
#h1n1_svm_multilabel_train_recall = recall_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])
#h1n1_svm_multilabel_train_precision = precision_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])
#h1n1_svm_multilabel_train_f1 = f1_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])
#h1n1_svm_multilabel_train_roc = roc_auc_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])

#print("Accuracy: {:.2f}".format(accuracy_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])))
#print("Recall: {:.2f}".format(recall_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])))
#print("Precision: {:.2f}".format(precision_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])))
#print("F1: {:.2f}".format(f1_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])))
#print("ROC: {:.2f}".format(roc_auc_score(y_both_vacc_train[:, 0], svm_multilabel_trainpreds[:, 0])))

In [ ]:
# SVM--Train data Evaluation Metrics for seasonal Vaccines
#seasonal_svm_multilabel_train_acc = accuracy_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])
#seasonal_svm_multilabel_train_recall = recall_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])
#seasonal_svm_multilabel_train_precision = precision_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])
#seasonal_svm_multilabel_train_f1 = f1_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])
#seasonal_svm_multilabel_train_roc = roc_auc_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])

#print("Accuracy: {:.2f}".format(accuracy_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])))
#print("Recall: {:.2f}".format(recall_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])))
#print("Precision: {:.2f}".format(precision_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])))
#print("F1: {:.2f}".format(f1_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])))
#print("ROC: {:.2f}".format(roc_auc_score(y_both_vacc_train[:, 1], svm_multilabel_trainpreds[:, 1])))

#### Test data

In [38]:
# Random Forest--Test data evaluation Metrics for H1N1 Vaccines
# THESE RESULTS ARE SLIGHTLY OFF FROM PREVIOUS NOTEBOOK (BY 0.01)--CHECK WHAT'S AT PLAY. VARIATION COMING FROM TEST-TRAIN SPLIT? EXPORT TEST-TRAIN DATA AS CSV AND TRY AGAIN
#IS IT BECAUSE IT'S RANDOM FOREST? DIFFERENT STARTING STUMP EACH TIME?
h1n1_rand_forst_multilabel_test_acc_best = accuracy_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds_best[:, 0])
h1n1_rand_forst_multilabel_test_recall_best = recall_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds_best[:, 0])
h1n1_rand_forst_multilabel_test_precision_best = precision_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds_best[:, 0])
h1n1_rand_forst_multilabel_test_f1_best = f1_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds_best[:, 0])
h1n1_rand_forst_multilabel_test_roc_best = roc_auc_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds_best[:, 0])

#print("Accuracy: {:.2f}".format(accuracy_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds[:, 0])))
#print("Recall: {:.2f}".format(recall_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds[:, 0])))
#print("Precision: {:.2f}".format(precision_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds[:, 0])))
#print("F1: {:.2f}".format(f1_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds[:, 0])))
#print("ROC: {:.2f}".format(roc_auc_score(y_both_vacc_test[:, 0], rand_forst_multilabel_testpreds[:, 0])))

In [39]:
# Random Forest--Test data Evaluation Metrics for seasonal Vaccines
# THESE RESULTS ARE SLIGHTLY OFF FROM PREVIOUS NOTEBOOK (BY 0.01)--CHECK WHAT'S AT PLAY. VARIATION COMING FROM TEST-TRAIN SPLIT? EXPORT TEST-TRAIN DATA AS CSV AND TRY AGAIN
#IS IT BECAUSE IT'S RANDOM FOREST? DIFFERENT STARTING STUMP EACH TIME?
seasonal_rand_forst_multilabel_test_acc_best = accuracy_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds_best[:, 1])
seasonal_rand_forst_multilabel_test_recall_best = recall_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds_best[:, 1])
seasonal_rand_forst_multilabel_test_precision_best = precision_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds_best[:, 1])
seasonal_rand_forst_multilabel_test_f1_best = f1_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds_best[:, 1])
seasonal_rand_forst_multilabel_test_roc_best = roc_auc_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds_best[:, 1])

#print("Accuracy: {:.2f}".format(accuracy_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds[:, 1])))
#print("Recall: {:.2f}".format(recall_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds[:, 1])))
#print("Precision: {:.2f}".format(precision_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds[:, 1])))
#print("F1: {:.2f}".format(f1_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds[:, 1])))
#print("ROC: {:.2f}".format(roc_auc_score(y_both_vacc_test[:, 1], rand_forst_multilabel_testpreds[:, 1])))

In [ ]:
# SVM--Test data evaluation Metrics for H1N1 Vaccines
#h1n1_svm_multilabel_test_acc = accuracy_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])
#h1n1_svm_multilabel_test_recall = recall_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])
#h1n1_svm_multilabel_test_precision = precision_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])
#h1n1_svm_multilabel_test_f1 = f1_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])
#h1n1_svm_multilabel_test_roc = roc_auc_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])

#print("Accuracy: {:.2f}".format(accuracy_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])))
#print("Recall: {:.2f}".format(recall_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])))
#print("Precision: {:.2f}".format(precision_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])))
#print("F1: {:.2f}".format(f1_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])))
#print("ROC: {:.2f}".format(roc_auc_score(y_both_vacc_test[:, 0], svm_multilabel_testpreds[:, 0])))

In [ ]:
# SVM--Test data Evaluation Metrics for seasonal Vaccines
#seasonal_svm_multilabel_test_acc = accuracy_score(y_both_vacc_test[:, 1], svm_multilabel_testpreds[:, 1])
#seasonal_svm_multilabel_test_recall = recall_score(y_both_vacc_test[:, 1], svm_multilabel_testpreds[:, 1])
#seasonal_svm_multilabel_test_precision = precision_score(y_both_vacc_test[:, 1], svm_multilabel_testpreds[:, 1])
#seasonal_svm_multilabel_test_f1 = f1_score(y_both_vacc_test[:, 1], svm_multilabel_testpreds[:, 1])
#seasonal_svm_multilabel_test_roc = roc_auc_score(y_both_vacc_test[:, 1], svm_multilabel_testpreds[:, 1])

#print("Accuracy: {:.2f}".format(accuracy_score(y_both_vacc_test[:, 1], logreg_multilabel_testpreds[:, 1])))
#print("Recall: {:.2f}".format(recall_score(y_both_vacc_test[:, 1], logreg_multilabel_testpreds[:, 1])))
#print("Precision: {:.2f}".format(precision_score(y_both_vacc_test[:, 1], logreg_multilabel_testpreds[:, 1])))
#print("F1: {:.2f}".format(f1_score(y_both_vacc_test[:, 1], logreg_multilabel_testpreds[:, 1])))
#print("ROC: {:.2f}".format(roc_auc_score(y_both_vacc_test[:, 1], logreg_multilabel_testpreds[:, 1])))

## Tracking the model with MLFlow

### H1N1 vaccine output

#### Random forest

In [40]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_multilabel)
name = 'rand_forst_multilabel_h1n1' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [41]:
print("Active run_id: {}".format(run.info.run_id))

Active run_id: bcad068b0db144189d86a1a4b784a98b


In [43]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": {'max_depth': 11, 
                        'max_features':0.5,
                        'max_leaf_nodes':45,
                         'n_estimators':94}
  }

In [44]:
# updated 
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", h1n1_rand_forst_multilabel_train_roc_best)
mlflow.log_metric("test -" + "ROC", h1n1_rand_forst_multilabel_test_roc_best)
mlflow.log_metric("train -" + "accuracy", h1n1_rand_forst_multilabel_train_acc_best)
mlflow.log_metric("test -" + "accuracy", h1n1_rand_forst_multilabel_test_acc_best)
mlflow.log_metric("train -" + "recall", h1n1_rand_forst_multilabel_train_recall_best)
mlflow.log_metric("test -" + "recall", h1n1_rand_forst_multilabel_test_recall_best)
mlflow.log_metric("train -" + "precision", h1n1_rand_forst_multilabel_train_precision_best)
mlflow.log_metric("test -" + "precision", h1n1_rand_forst_multilabel_test_precision_best)
mlflow.log_metric("train -" + "f1", h1n1_rand_forst_multilabel_train_f1_best)
mlflow.log_metric("test -" + "f1", h1n1_rand_forst_multilabel_test_f1_best)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### SVM

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_multilabel)
name = 'svm_multilabel_h1n1' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", h1n1_svm_multilabel_train_roc)
mlflow.log_metric("test -" + "ROC", h1n1_svm_multilabel_test_roc)
mlflow.log_metric("train -" + "accuracy", h1n1_svm_multilabel_train_acc)
mlflow.log_metric("test -" + "accuracy", h1n1_svm_multilabel_test_acc)
mlflow.log_metric("train -" + "recall", h1n1_svm_multilabel_train_recall)
mlflow.log_metric("test -" + "recall", h1n1_svm_multilabel_test_recall)
mlflow.log_metric("train -" + "precision", h1n1_svm_multilabel_train_precision)
mlflow.log_metric("test -" + "precision", h1n1_svm_multilabel_test_precision)
mlflow.log_metric("train -" + "f1", h1n1_svm_multilabel_train_f1)
mlflow.log_metric("test -" + "f1", h1n1_svm_multilabel_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

### Seasonal vaccine output

#### Random forest

In [45]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_multilabel)
name = 'rand_forst_multilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [46]:
print("Active run_id: {}".format(run.info.run_id))

Active run_id: 57318dbf7f99461fbf12324dfd7233fd


In [47]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": {'max_depth': 11, 
                        'max_features':0.5,
                        'max_leaf_nodes':45,
                         'n_estimators':94}
  }

In [48]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seasonal_rand_forst_multilabel_train_roc_best)
mlflow.log_metric("test -" + "ROC", seasonal_rand_forst_multilabel_test_roc_best)
mlflow.log_metric("train -" + "accuracy", seasonal_rand_forst_multilabel_train_acc_best)
mlflow.log_metric("test -" + "accuracy", seasonal_rand_forst_multilabel_test_acc_best)
mlflow.log_metric("train -" + "recall", seasonal_rand_forst_multilabel_train_recall_best)
mlflow.log_metric("test -" + "recall", seasonal_rand_forst_multilabel_test_recall_best)
mlflow.log_metric("train -" + "precision", seasonal_rand_forst_multilabel_train_precision_best)
mlflow.log_metric("test -" + "precision", seasonal_rand_forst_multilabel_test_precision_best)
mlflow.log_metric("train -" + "f1", seasonal_rand_forst_multilabel_train_f1_best)
mlflow.log_metric("test -" + "f1", seasonal_rand_forst_multilabel_test_f1_best)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### SVM

In [ ]:
# setting the MLFlow connection and experiment
'''mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_multilabel)
name = 'svm_multilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()'''

In [ ]:
'''print("Active run_id: {}".format(run.info.run_id))'''

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
'''params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }'''

In [ ]:
'''mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seasonal_svm_multilabel_train_roc)
mlflow.log_metric("test -" + "ROC", seasonal_svm_multilabel_test_roc)
mlflow.log_metric("train -" + "accuracy", seasonal_svm_multilabel_train_acc)
mlflow.log_metric("test -" + "accuracy", seasonal_svm_multilabel_test_acc)
mlflow.log_metric("train -" + "recall", seasonal_svm_multilabel_train_recall)
mlflow.log_metric("test -" + "recall", seasonal_svm_multilabel_test_recall)
mlflow.log_metric("train -" + "precision", seasonal_svm_multilabel_train_precision)
mlflow.log_metric("test -" + "precision", seasonal_svm_multilabel_test_precision)
mlflow.log_metric("train -" + "f1", seasonal_svm_multilabel_train_f1)
mlflow.log_metric("test -" + "f1", seasonal_svm_multilabel_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()'''

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

# TheFluShot_H1N1: Single Label Modelling, output H1N1 vaccine

## Single Label Modelling, output H1N1 vaccine -> Seasonal Flu Vaccine not in features

The cat_features_no_vacc and X_no_vacc variables and the preprocessor remain the same from the multilabel modelling:

Set up the target variable:

In [46]:
y_h1n1_vacc = df[['h1n1_vaccine']].copy()

In [47]:
y_h1n1_vacc = y_h1n1_vacc.to_numpy()


In [48]:
y_h1n1_vacc = y_h1n1_vacc.ravel()

In [49]:
y_h1n1_vacc.shape

(26707,)

Performing test-train split:

In [50]:
X_no_vacc_train, X_no_vacc_test, y_h1n1_vacc_train, y_h1n1_vacc_test = train_test_split(X_no_vacc, y_h1n1_vacc, stratify = y_h1n1_vacc, test_size=0.2, random_state=RSEED)

In [51]:
print('X_no_vacc_train shape:', X_no_vacc_train.shape)
print('X_no_vacc_test shape:', X_no_vacc_test.shape)
print('y_h1n1_vacc_train:', y_h1n1_vacc_train.shape)
print('y_h1n1_vacc_test:', y_h1n1_vacc_test.shape)

X_no_vacc_train shape: (21365, 35)
X_no_vacc_test shape: (5342, 35)
y_h1n1_vacc_train: (21365,)
y_h1n1_vacc_test: (5342,)


Setting up the pipeline for each model:

In [52]:
# for Random Forest

rand_forst_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", rand_forst),
])

'''#for SVM
svm_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", svm),
])'''

'#for SVM\nsvm_unilabel_pipeline = Pipeline([\n    ("preprocessor", preprocessor),\n    ("estimators", svm),\n])'

### Setting up the Grid Search for hyperparameter tuning

In [53]:
rand_forst.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [54]:
# will need to be adjusted in further runs 
# the pipeline step is accessed with 'estimators__estimator__'

param_grid_unilabel = {
    'estimators__n_estimators': np.linspace(10, 100, 200).astype(int),
    'estimators__max_depth': [None] + list(np.linspace(3, 20).astype(int)),
    'estimators__max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'estimators__max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    #'estimators__estimator__min_samples_split': [2, 5, 10],
    'estimators__bootstrap': [True, False]
}

In [55]:
# the random search model is created 
# we define roc_auc as our evaluation metric for the parameter search 
# set verbose to a higher number next time 

grid_rand_forst_unilabel = RandomizedSearchCV(rand_forst_unilabel_pipeline, param_distributions=param_grid_unilabel, cv=5, scoring='roc_auc', 
                           verbose=50, n_jobs=-1, n_iter=100)

### Fitting the grid search to our model 

In [56]:
rand_forst_unilabel = grid_rand_forst_unilabel.fit(X_no_vacc_train,  y_h1n1_vacc_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 4/5; 1/100] START estimators__bootstrap=False, estimators__max_depth=7, estimators__max_features=0.7, estimators__max_leaf_nodes=28, estimators__n_estimators=13
[CV 2/5; 1/100] START estimators__bootstrap=False, estimators__max_depth=7, estimators__max_features=0.7, estimators__max_leaf_nodes=28, estimators__n_estimators=13
[CV 1/5; 1/100] START estimators__bootstrap=False, estimators__max_depth=7, estimators__max_features=0.7, estimators__max_leaf_nodes=28, estimators__n_estimators=13
[CV 3/5; 1/100] START estimators__bootstrap=False, estimators__max_depth=7, estimators__max_features=0.7, estimators__max_leaf_nodes=28, estimators__n_estimators=13
[CV 2/5; 1/100] END estimators__bootstrap=False, estimators__max_depth=7, estimators__max_features=0.7, estimators__max_leaf_nodes=28, estimators__n_estimators=13;, score=0.845 total time=  33.7s
[CV 3/5; 1/100] END estimators__bootstrap=False, estimators__max_depth=7, estimat

In [57]:
# Show best parameters
print('Best score:\n{:.2f}'.format(rand_forst_unilabel.best_score_))
print("Best parameters:\n{}".format(rand_forst_unilabel.best_params_))

Best score:
0.85
Best parameters:
{'estimators__n_estimators': 67, 'estimators__max_leaf_nodes': 48, 'estimators__max_features': 'auto', 'estimators__max_depth': 16, 'estimators__bootstrap': True}


In [58]:
# Save best model

best_model_rand_forst_unilabel = rand_forst_unilabel.best_estimator_['estimators']

In [59]:
best_model_rand_forst_unilabel

RandomForestClassifier(max_depth=16, max_leaf_nodes=48, n_estimators=67)

In [60]:
# Creating an adapted pipeline 

rand_forst_unilabel_pipeline_tuned = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", best_model_rand_forst_unilabel),
])

Training the optimized model on the data 

In [61]:
# for Random Forest
# fitting the data to the new tuned model 

best_model_rand_forst_unilabel = rand_forst_unilabel_pipeline_tuned.fit(X_no_vacc_train,  y_h1n1_vacc_train)

#for SVM
'''svm_unilabel_no_vacc = svm_unilabel_pipeline.fit(X_no_vacc_train, y_h1n1_vacc_train)'''

'svm_unilabel_no_vacc = svm_unilabel_pipeline.fit(X_no_vacc_train, y_h1n1_vacc_train)'

In [ ]:
# Figure out later what this does and if we want to use it


#y_train_predicted = cross_val_predict(full_pipeline, X_train, y_train, cv=5)

Making predictions based on train and test data:

In [62]:
# for Random Forest
rand_forst_unilabel_no_vacc_trainpreds_best = best_model_rand_forst_unilabel.predict(X_no_vacc_train)
rand_forst_unilabel_no_vacc_testpreds_best = best_model_rand_forst_unilabel.predict(X_no_vacc_test)

# for SVM
#svm_unilabel_no_vacc_trainpreds = svm_unilabel_no_vacc.predict(X_no_vacc_train)
#svm_unilabel_no_vacc_testpreds = svm_unilabel_no_vacc.predict(X_no_vacc_test)

### Model evaluation

#### Train data

In [63]:
# Random Forest--Train data evaluation Metrics
#check if these metrics are changing compared to previous results
h1n1_rand_forst_unilabel_no_vacc_train_acc_best = accuracy_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds_best)
h1n1_rand_forst_unilabel_no_vacc_train_recall_best = recall_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds_best)
h1n1_rand_forst_unilabel_no_vacc_train_precision_best = precision_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds_best)
h1n1_rand_forst_unilabel_no_vacc_train_f1_best = f1_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds_best)
h1n1_rand_forst_unilabel_no_vacc_train_roc_best = roc_auc_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds_best)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))

In [ ]:
# SVM--Train data evaluation Metrics
'''h1n1_svm_unilabel_no_vacc_train_acc = accuracy_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)
h1n1_svm_unilabel_no_vacc_train_recall = recall_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)
h1n1_svm_unilabel_no_vacc_train_precision = precision_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)
h1n1_svm_unilabel_no_vacc_train_f1 = f1_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)
h1n1_svm_unilabel_no_vacc_train_roc = roc_auc_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)'''

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_train, svm_unilabel_no_vacc_trainpreds)))

#### Test data

In [64]:
# Random Forest--Test data evaluation Metrics
#check if these metrics are changing compared to previous results
h1n1_rand_forst_unilabel_no_vacc_test_acc_best = accuracy_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds_best)
h1n1_rand_forst_unilabel_no_vacc_test_recall_best = recall_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds_best)
h1n1_rand_forst_unilabel_no_vacc_test_precision_best = precision_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds_best)
h1n1_rand_forst_unilabel_no_vacc_test_f1_best = f1_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds_best)
h1n1_rand_forst_unilabel_no_vacc_test_roc_best = roc_auc_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds_best)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))

In [ ]:
# SVM--Test data evaluation Metrics
'''h1n1_svm_unilabel_no_vacc_test_acc = accuracy_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)
h1n1_svm_unilabel_no_vacc_test_recall = recall_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)
h1n1_svm_unilabel_no_vacc_test_precision = precision_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)
h1n1_svm_unilabel_no_vacc_test_f1 = f1_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)
h1n1_svm_unilabel_no_vacc_test_roc = roc_auc_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)'''

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_test, svm_unilabel_no_vacc_testpreds)))

## Tracking the model with MLFlow

### H1N1 vaccine output

#### Random forest

In [65]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_h1n1)
name = 'rand_forst_unilabel_h1n1' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [66]:
print("Active run_id: {}".format(run.info.run_id))

Active run_id: c76d90eda22a4576922e621259c2a550


In [67]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": {'estimators__n_estimators': 67, 'estimators__max_leaf_nodes': 48, 'estimators__max_features': 'auto', 'estimators__max_depth': 16, 'estimators__bootstrap': True}
  }

In [68]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", h1n1_rand_forst_unilabel_no_vacc_train_roc_best)
mlflow.log_metric("test -" + "ROC", h1n1_rand_forst_unilabel_no_vacc_test_roc_best)
mlflow.log_metric("train -" + "accuracy", h1n1_rand_forst_unilabel_no_vacc_train_acc_best)
mlflow.log_metric("test -" + "accuracy", h1n1_rand_forst_unilabel_no_vacc_test_acc_best)
mlflow.log_metric("train -" + "recall", h1n1_rand_forst_unilabel_no_vacc_train_recall_best)
mlflow.log_metric("test -" + "recall", h1n1_rand_forst_unilabel_no_vacc_test_recall_best)
mlflow.log_metric("train -" + "precision", h1n1_rand_forst_unilabel_no_vacc_train_precision_best)
mlflow.log_metric("test -" + "precision", h1n1_rand_forst_unilabel_no_vacc_test_precision_best)
mlflow.log_metric("train -" + "f1", h1n1_rand_forst_unilabel_no_vacc_train_f1_best)
mlflow.log_metric("test -" + "f1", h1n1_rand_forst_unilabel_no_vacc_test_f1_best)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### SVM

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_h1n1)
name = 'svm_unilabel_h1n1' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", h1n1_svm_unilabel_no_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", h1n1_svm_unilabel_no_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", h1n1_svm_unilabel_no_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", h1n1_svm_unilabel_no_vacc_test_acc)
mlflow.log_metric("train -" + "recall", h1n1_svm_unilabel_no_vacc_train_recall)
mlflow.log_metric("test -" + "recall", h1n1_svm_unilabel_no_vacc_test_recall)
mlflow.log_metric("train -" + "precision", h1n1_svm_unilabel_no_vacc_train_precision)
mlflow.log_metric("test -" + "precision", h1n1_svm_unilabel_no_vacc_test_precision)
mlflow.log_metric("train -" + "f1", h1n1_svm_unilabel_no_vacc_train_f1)
mlflow.log_metric("test -" + "f1", h1n1_svm_unilabel_no_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

## Single Label Modelling, output H1N1 vaccine -> Seasonal Flu Vaccine is in features

The y_h1n1_vacc remains the same from the previous model; the X feature and cat_features (for the preprocessor) need to be adjusted:

In [ ]:
cat_features_seas_vacc = cat_features.copy()

In [ ]:
cat_features_seas_vacc.remove('h1n1_vaccine')

In [ ]:
#NB: dropping the 'h1n1_vaccine' and 'seasonal_vaccine' columns
X_seas_vacc = df.drop(columns=['h1n1_vaccine'])

In [ ]:
X_seas_vacc.columns

Performing test-train split (the same data can be used for each model in multilabelling):

In [ ]:
X_seas_vacc_train, X_seas_vacc_test, y_h1n1_vacc_train, y_h1n1_vacc_test = train_test_split(X_seas_vacc, y_h1n1_vacc, stratify = y_h1n1_vacc, test_size=0.2, random_state=RSEED)

In [ ]:
print('X_seas_vacc_train shape:', X_seas_vacc_train.shape)
print('X_seas_vacc_test shape:', X_seas_vacc_test.shape)
print('y_h1n1_vacc_train:', y_h1n1_vacc_train.shape)
print('y_h1n1_vacc_test:', y_h1n1_vacc_test.shape)

Preprocessor is adjusted:

In [ ]:
preprocessor_seas_vacc = ColumnTransformer([
    ('cat', cat_pipeline, cat_features_seas_vacc)
])

Pipeline is adjusted:

In [ ]:
# for logreg
logreg_seas_vacc_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor_seas_vacc),
    ("estimators", logreg),
])

# for KNN
knn_seas_vacc_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor_seas_vacc),
    ("estimators", knn),
])

# for Random Forest

rand_forst_seas_vacc_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor_seas_vacc),
    ("estimators", rand_forst),
])

#for SVM
svm_seas_vacc_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor_seas_vacc),
    ("estimators", svm),
])

Training the models:

In [ ]:
# for logreg
logreg_unilabel_seas_vacc = logreg_seas_vacc_unilabel_pipeline.fit(X_seas_vacc_train, y_h1n1_vacc_train)

# for KNN
knn_unilabel_seas_vacc = knn_seas_vacc_unilabel_pipeline.fit(X_seas_vacc_train, y_h1n1_vacc_train)

# for Random Forest
rand_forst_unilabel_seas_vacc = rand_forst_seas_vacc_unilabel_pipeline.fit(X_seas_vacc_train, y_h1n1_vacc_train)

#for SVM
svm_unilabel_seas_vacc = svm_seas_vacc_unilabel_pipeline.fit(X_seas_vacc_train, y_h1n1_vacc_train)

In [ ]:
# Figure out later what this does and if we want to use it


#y_train_predicted = cross_val_predict(full_pipeline, X_train, y_train, cv=5)

Making predictions based on train and test data:

In [ ]:
# for logreg
logreg_unilabel_seas_vacc_trainpreds = logreg_unilabel_seas_vacc.predict(X_seas_vacc_train)
logreg_unilabel_seas_vacc_testpreds = logreg_unilabel_seas_vacc.predict(X_seas_vacc_test)

# for KNN
knn_unilabel_seas_vacc_trainpreds = knn_unilabel_seas_vacc.predict(X_seas_vacc_train)
knn_unilabel_seas_vacc_testpreds = knn_unilabel_seas_vacc.predict(X_seas_vacc_test)

# for Random Forest
rand_forst_unilabel_seas_vacc_trainpreds = rand_forst_unilabel_seas_vacc.predict(X_seas_vacc_train)
rand_forst_unilabel_seas_vacc_testpreds = rand_forst_unilabel_seas_vacc.predict(X_seas_vacc_test)

# for SVM
svm_unilabel_seas_vacc_trainpreds = svm_unilabel_seas_vacc.predict(X_seas_vacc_train)
svm_unilabel_seas_vacc_testpreds = svm_unilabel_seas_vacc.predict(X_seas_vacc_test)

### Model evaluation

#### Train data

In [ ]:
# Logreg--Train data evaluation Metrics
h1n1_logreg_unilabel_seas_vacc_train_acc = accuracy_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)
h1n1_logreg_unilabel_seas_vacc_train_recall = recall_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)
h1n1_logreg_unilabel_seas_vacc_train_precision = precision_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)
h1n1_logreg_unilabel_seas_vacc_train_f1 = f1_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)
h1n1_logreg_unilabel_seas_vacc_train_roc = roc_auc_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_train, logreg_unilabel_seas_vacc_trainpreds)))

In [ ]:
# KNN--Train data evaluation Metrics
h1n1_knn_unilabel_seas_vacc_train_acc = accuracy_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)
h1n1_knn_unilabel_seas_vacc_train_recall = recall_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)
h1n1_knn_unilabel_seas_vacc_train_precision = precision_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)
h1n1_knn_unilabel_seas_vacc_train_f1 = f1_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)
h1n1_knn_unilabel_seas_vacc_train_roc = roc_auc_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_train, knn_unilabel_seas_vacc_trainpreds)))

In [ ]:
# Random Forest--Train data evaluation Metrics
#check if these metrics are changing compared to previous results
h1n1_rand_forst_unilabel_seas_vacc_train_acc = accuracy_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)
h1n1_rand_forst_unilabel_seas_vacc_train_recall = recall_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)
h1n1_rand_forst_unilabel_seas_vacc_train_precision = precision_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)
h1n1_rand_forst_unilabel_seas_vacc_train_f1 = f1_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)
h1n1_rand_forst_unilabel_seas_vacc_train_roc = roc_auc_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_train, rand_forst_unilabel_seas_vacc_trainpreds)))

In [ ]:
# SVM--Train data evaluation Metrics
h1n1_svm_unilabel_seas_vacc_train_acc = accuracy_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)
h1n1_svm_unilabel_seas_vacc_train_recall = recall_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)
h1n1_svm_unilabel_seas_vacc_train_precision = precision_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)
h1n1_svm_unilabel_seas_vacc_train_f1 = f1_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)
h1n1_svm_unilabel_seas_vacc_train_roc = roc_auc_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_train, svm_unilabel_seas_vacc_trainpreds)))

#### Test data

In [ ]:
# Logreg--Test data evaluation Metrics
h1n1_logreg_unilabel_seas_vacc_test_acc = accuracy_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)
h1n1_logreg_unilabel_seas_vacc_test_recall = recall_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)
h1n1_logreg_unilabel_seas_vacc_test_precision = precision_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)
h1n1_logreg_unilabel_seas_vacc_test_f1 = f1_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)
h1n1_logreg_unilabel_seas_vacc_test_roc = roc_auc_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_test, logreg_unilabel_seas_vacc_testpreds)))

In [ ]:
# KNN--Test data evaluation Metrics
h1n1_knn_unilabel_seas_vacc_test_acc = accuracy_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)
h1n1_knn_unilabel_seas_vacc_test_recall = recall_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)
h1n1_knn_unilabel_seas_vacc_test_precision = precision_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)
h1n1_knn_unilabel_seas_vacc_test_f1 = f1_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)
h1n1_knn_unilabel_seas_vacc_test_roc = roc_auc_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_test, knn_unilabel_seas_vacc_testpreds)))

In [ ]:
# Random Forest--Test data evaluation Metrics
#check if these metrics are changing compared to previous results
h1n1_rand_forst_unilabel_seas_vacc_test_acc = accuracy_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)
h1n1_rand_forst_unilabel_seas_vacc_test_recall = recall_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)
h1n1_rand_forst_unilabel_seas_vacc_test_precision = precision_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)
h1n1_rand_forst_unilabel_seas_vacc_test_f1 = f1_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)
h1n1_rand_forst_unilabel_seas_vacc_test_roc = roc_auc_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_test, rand_forst_unilabel_seas_vacc_testpreds)))

In [ ]:
# SVM--Test data evaluation Metrics
h1n1_svm_unilabel_seas_vacc_test_acc = accuracy_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)
h1n1_svm_unilabel_seas_vacc_test_recall = recall_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)
h1n1_svm_unilabel_seas_vacc_test_precision = precision_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)
h1n1_svm_unilabel_seas_vacc_test_f1 = f1_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)
h1n1_svm_unilabel_seas_vacc_test_roc = roc_auc_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_h1n1_vacc_test, svm_unilabel_seas_vacc_testpreds)))

## Tracking the model with MLFlow

### H1N1 vaccine output

#### Logistic regression

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_h1n1) #this needs to be adjusted for each experiment
name = 'logreg_unilabel_h1n1' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "Seasonal") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", h1n1_logreg_unilabel_seas_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", h1n1_logreg_unilabel_seas_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", h1n1_logreg_unilabel_seas_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", h1n1_logreg_unilabel_seas_vacc_test_acc)
mlflow.log_metric("train -" + "recall", h1n1_logreg_unilabel_seas_vacc_train_recall)
mlflow.log_metric("test -" + "recall", h1n1_logreg_unilabel_seas_vacc_test_recall)
mlflow.log_metric("train -" + "precision", h1n1_logreg_unilabel_seas_vacc_train_precision)
mlflow.log_metric("test -" + "precision", h1n1_logreg_unilabel_seas_vacc_test_precision)
mlflow.log_metric("train -" + "f1", h1n1_logreg_unilabel_seas_vacc_train_f1)
mlflow.log_metric("test -" + "f1", h1n1_logreg_unilabel_seas_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### KNN

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_h1n1)
name = 'knn_unilabel_h1n1' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "Seasonal") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", h1n1_knn_unilabel_seas_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", h1n1_knn_unilabel_seas_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", h1n1_knn_unilabel_seas_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", h1n1_knn_unilabel_seas_vacc_test_acc)
mlflow.log_metric("train -" + "recall", h1n1_knn_unilabel_seas_vacc_train_recall)
mlflow.log_metric("test -" + "recall", h1n1_knn_unilabel_seas_vacc_test_recall)
mlflow.log_metric("train -" + "precision", h1n1_knn_unilabel_seas_vacc_train_precision)
mlflow.log_metric("test -" + "precision", h1n1_knn_unilabel_seas_vacc_test_precision)
mlflow.log_metric("train -" + "f1", h1n1_knn_unilabel_seas_vacc_train_f1)
mlflow.log_metric("test -" + "f1", h1n1_knn_unilabel_seas_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### Random forest

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_h1n1)
name = 'rand_forst_unilabel_h1n1' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "Seasonal") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", h1n1_rand_forst_unilabel_seas_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", h1n1_rand_forst_unilabel_seas_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", h1n1_rand_forst_unilabel_seas_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", h1n1_rand_forst_unilabel_seas_vacc_test_acc)
mlflow.log_metric("train -" + "recall", h1n1_rand_forst_unilabel_seas_vacc_train_recall)
mlflow.log_metric("test -" + "recall", h1n1_rand_forst_unilabel_seas_vacc_test_recall)
mlflow.log_metric("train -" + "precision", h1n1_rand_forst_unilabel_seas_vacc_train_precision)
mlflow.log_metric("test -" + "precision", h1n1_rand_forst_unilabel_seas_vacc_test_precision)
mlflow.log_metric("train -" + "f1", h1n1_rand_forst_unilabel_seas_vacc_train_f1)
mlflow.log_metric("test -" + "f1", h1n1_rand_forst_unilabel_seas_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### SVM

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_h1n1)
name = 'svm_unilabel_h1n1' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "Seasonal") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", h1n1_svm_unilabel_seas_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", h1n1_svm_unilabel_seas_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", h1n1_svm_unilabel_seas_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", h1n1_svm_unilabel_seas_vacc_test_acc)
mlflow.log_metric("train -" + "recall", h1n1_svm_unilabel_seas_vacc_train_recall)
mlflow.log_metric("test -" + "recall", h1n1_svm_unilabel_seas_vacc_test_recall)
mlflow.log_metric("train -" + "precision", h1n1_svm_unilabel_seas_vacc_train_precision)
mlflow.log_metric("test -" + "precision", h1n1_svm_unilabel_seas_vacc_test_precision)
mlflow.log_metric("train -" + "f1", h1n1_svm_unilabel_seas_vacc_train_f1)
mlflow.log_metric("test -" + "f1", h1n1_svm_unilabel_seas_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

# TheFluShot_seasonal: Single Label Modelling, output seasonal vaccine

## Single Label Modelling, output seasonal vaccine -> H1N1 Flu Vaccine not in features

The cat_features_no_vacc and X_no_vacc variables and the preprocessor remain the same from the multilabel modelling:

Set up the target variable:

In [ ]:
y_seas_vacc = df['seasonal_vaccine'].copy()

In [ ]:
y_seas_vacc = y_seas_vacc.to_numpy()
y_seas_vacc

In [ ]:
len(y_seas_vacc)

Performing test-train split:

In [ ]:
X_no_vacc_train, X_no_vacc_test, y_seas_vacc_train, y_seas_vacc_test = train_test_split(X_no_vacc, y_seas_vacc, stratify = y_seas_vacc, test_size=0.2, random_state=RSEED)

In [ ]:
print('X_no_vacc_train shape:', X_no_vacc_train.shape)
print('X_no_vacc_test shape:', X_no_vacc_test.shape)
print('y_seas_vacc_train:', y_seas_vacc_train.shape)
print('y_seas_vacc_test:', y_seas_vacc_test.shape)

Setting up the pipeline for each model:

In [ ]:
# for logreg
logreg_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", logreg),
])

# for KNN
knn_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", knn),
])

# for Random Forest

rand_forst_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", rand_forst),
])

#for SVM
svm_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", svm),
])

Training the models:

In [ ]:
# for logreg
logreg_unilabel_no_vacc = logreg_unilabel_pipeline.fit(X_no_vacc_train, y_seas_vacc_train)

# for KNN
knn_unilabel_no_vacc = knn_unilabel_pipeline.fit(X_no_vacc_train, y_seas_vacc_train)

# for Random Forest
rand_forst_unilabel_no_vacc = rand_forst_unilabel_pipeline.fit(X_no_vacc_train, y_seas_vacc_train)

#for SVM
svm_unilabel_no_vacc = svm_unilabel_pipeline.fit(X_no_vacc_train, y_seas_vacc_train)

In [ ]:
# Figure out later what this does and if we want to use it


#y_train_predicted = cross_val_predict(full_pipeline, X_train, y_train, cv=5)

Making predictions based on train and test data:

In [ ]:
# for logreg
logreg_unilabel_no_vacc_trainpreds = logreg_unilabel_no_vacc.predict(X_no_vacc_train)
logreg_unilabel_no_vacc_testpreds = logreg_unilabel_no_vacc.predict(X_no_vacc_test)

# for KNN
knn_unilabel_no_vacc_trainpreds = knn_unilabel_no_vacc.predict(X_no_vacc_train)
knn_unilabel_no_vacc_testpreds = knn_unilabel_no_vacc.predict(X_no_vacc_test)

# for Random Forest
rand_forst_unilabel_no_vacc_trainpreds = rand_forst_unilabel_no_vacc.predict(X_no_vacc_train)
rand_forst_unilabel_no_vacc_testpreds = rand_forst_unilabel_no_vacc.predict(X_no_vacc_test)

# for SVM
svm_unilabel_no_vacc_trainpreds = svm_unilabel_no_vacc.predict(X_no_vacc_train)
svm_unilabel_no_vacc_testpreds = svm_unilabel_no_vacc.predict(X_no_vacc_test)

### Model evaluation

#### Train data

In [ ]:
# Logreg--Train data evaluation Metrics
seas_logreg_unilabel_no_vacc_train_acc = accuracy_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)
seas_logreg_unilabel_no_vacc_train_recall = recall_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)
seas_logreg_unilabel_no_vacc_train_precision = precision_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)
seas_logreg_unilabel_no_vacc_train_f1 = f1_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)
seas_logreg_unilabel_no_vacc_train_roc = roc_auc_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_train, logreg_unilabel_no_vacc_trainpreds)))

In [ ]:
# KNN--Train data evaluation Metrics
seas_knn_unilabel_no_vacc_train_acc = accuracy_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)
seas_knn_unilabel_no_vacc_train_recall = recall_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)
seas_knn_unilabel_no_vacc_train_precision = precision_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)
seas_knn_unilabel_no_vacc_train_f1 = f1_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)
seas_knn_unilabel_no_vacc_train_roc = roc_auc_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_train, knn_unilabel_no_vacc_trainpreds)))

In [ ]:
# Random Forest--Train data evaluation Metrics
#check if these metrics are changing compared to previous results
seas_rand_forst_unilabel_no_vacc_train_acc = accuracy_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)
seas_rand_forst_unilabel_no_vacc_train_recall = recall_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)
seas_rand_forst_unilabel_no_vacc_train_precision = precision_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)
seas_rand_forst_unilabel_no_vacc_train_f1 = f1_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)
seas_rand_forst_unilabel_no_vacc_train_roc = roc_auc_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_train, rand_forst_unilabel_no_vacc_trainpreds)))

In [ ]:
# SVM--Train data evaluation Metrics
seas_svm_unilabel_no_vacc_train_acc = accuracy_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)
seas_svm_unilabel_no_vacc_train_recall = recall_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)
seas_svm_unilabel_no_vacc_train_precision = precision_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)
seas_svm_unilabel_no_vacc_train_f1 = f1_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)
seas_svm_unilabel_no_vacc_train_roc = roc_auc_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_train, svm_unilabel_no_vacc_trainpreds)))

#### Test data

In [ ]:
# Logreg--Test data evaluation Metrics
seas_logreg_unilabel_no_vacc_test_acc = accuracy_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)
seas_logreg_unilabel_no_vacc_test_recall = recall_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)
seas_logreg_unilabel_no_vacc_test_precision = precision_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)
seas_logreg_unilabel_no_vacc_test_f1 = f1_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)
seas_logreg_unilabel_no_vacc_test_roc = roc_auc_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_test, logreg_unilabel_no_vacc_testpreds)))

In [ ]:
# KNN--Test data evaluation Metrics
seas_knn_unilabel_no_vacc_test_acc = accuracy_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)
seas_knn_unilabel_no_vacc_test_recall = recall_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)
seas_knn_unilabel_no_vacc_test_precision = precision_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)
seas_knn_unilabel_no_vacc_test_f1 = f1_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)
seas_knn_unilabel_no_vacc_test_roc = roc_auc_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)

print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)))
print("Recall: {:.2f}".format(recall_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)))
print("Precision: {:.2f}".format(precision_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)))
print("F1: {:.2f}".format(f1_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)))
print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_test, knn_unilabel_no_vacc_testpreds)))

In [ ]:
print(seas_knn_unilabel_no_vacc_test_acc)
print(seas_knn_unilabel_no_vacc_test_recall)
print(seas_knn_unilabel_no_vacc_test_precision)
print(seas_knn_unilabel_no_vacc_test_f1)
print(seas_knn_unilabel_no_vacc_test_roc)

In [ ]:
# Random Forest--Test data evaluation Metrics
#check if these metrics are changing compared to previous results
seas_rand_forst_unilabel_no_vacc_test_acc = accuracy_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)
seas_rand_forst_unilabel_no_vacc_test_recall = recall_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)
seas_rand_forst_unilabel_no_vacc_test_precision = precision_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)
seas_rand_forst_unilabel_no_vacc_test_f1 = f1_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)
seas_rand_forst_unilabel_no_vacc_test_roc = roc_auc_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)

print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))
print("Recall: {:.2f}".format(recall_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))
print("Precision: {:.2f}".format(precision_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))
print("F1: {:.2f}".format(f1_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))
print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_test, rand_forst_unilabel_no_vacc_testpreds)))

In [ ]:
print(seas_rand_forst_unilabel_no_vacc_test_acc)
print(seas_rand_forst_unilabel_no_vacc_test_recall)
print(seas_rand_forst_unilabel_no_vacc_test_precision)
print(seas_rand_forst_unilabel_no_vacc_test_f1)
print(seas_rand_forst_unilabel_no_vacc_test_roc)

In [ ]:
# SVM--Test data evaluation Metrics
seas_svm_unilabel_no_vacc_test_acc = accuracy_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)
seas_svm_unilabel_no_vacc_test_recall = recall_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)
seas_svm_unilabel_no_vacc_test_precision = precision_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)
seas_svm_unilabel_no_vacc_test_f1 = f1_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)
seas_svm_unilabel_no_vacc_test_roc = roc_auc_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_test, svm_unilabel_no_vacc_testpreds)))

## Tracking the model with MLFlow

### Seasonal vaccine output

#### Logistic regression

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_seasonal) #this needs to be adjusted for each experiment
name = 'logreg_unilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seas_logreg_unilabel_no_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", seas_logreg_unilabel_no_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", seas_logreg_unilabel_no_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", seas_logreg_unilabel_no_vacc_test_acc)
mlflow.log_metric("train -" + "recall", seas_logreg_unilabel_no_vacc_train_recall)
mlflow.log_metric("test -" + "recall", seas_logreg_unilabel_no_vacc_test_recall)
mlflow.log_metric("train -" + "precision", seas_logreg_unilabel_no_vacc_train_precision)
mlflow.log_metric("test -" + "precision", seas_logreg_unilabel_no_vacc_test_precision)
mlflow.log_metric("train -" + "f1", seas_logreg_unilabel_no_vacc_train_f1)
mlflow.log_metric("test -" + "f1", seas_logreg_unilabel_no_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### KNN

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_seasonal)
name = 'knn_unilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seas_knn_unilabel_no_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", seas_knn_unilabel_no_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", seas_knn_unilabel_no_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", seas_knn_unilabel_no_vacc_test_acc)
mlflow.log_metric("train -" + "recall", seas_knn_unilabel_no_vacc_train_recall)
mlflow.log_metric("test -" + "recall", seas_knn_unilabel_no_vacc_test_recall)
mlflow.log_metric("train -" + "precision", seas_knn_unilabel_no_vacc_train_precision)
mlflow.log_metric("test -" + "precision", seas_knn_unilabel_no_vacc_test_precision)
mlflow.log_metric("train -" + "f1", seas_knn_unilabel_no_vacc_train_f1)
mlflow.log_metric("test -" + "f1", seas_knn_unilabel_no_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### Random forest

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_seasonal)
name = 'rand_forst_unilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seas_rand_forst_unilabel_no_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", seas_rand_forst_unilabel_no_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", seas_rand_forst_unilabel_no_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", seas_rand_forst_unilabel_no_vacc_test_acc)
mlflow.log_metric("train -" + "recall", seas_rand_forst_unilabel_no_vacc_train_recall)
mlflow.log_metric("test -" + "recall", seas_rand_forst_unilabel_no_vacc_test_recall)
mlflow.log_metric("train -" + "precision", seas_rand_forst_unilabel_no_vacc_train_precision)
mlflow.log_metric("test -" + "precision", seas_rand_forst_unilabel_no_vacc_test_precision)
mlflow.log_metric("train -" + "f1", seas_rand_forst_unilabel_no_vacc_train_f1)
mlflow.log_metric("test -" + "f1", seas_rand_forst_unilabel_no_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### SVM

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_seasonal)
name = 'svm_unilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "None") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seas_svm_unilabel_no_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", seas_svm_unilabel_no_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", seas_svm_unilabel_no_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", seas_svm_unilabel_no_vacc_test_acc)
mlflow.log_metric("train -" + "recall", seas_svm_unilabel_no_vacc_train_recall)
mlflow.log_metric("test -" + "recall", seas_svm_unilabel_no_vacc_test_recall)
mlflow.log_metric("train -" + "precision", seas_svm_unilabel_no_vacc_train_precision)
mlflow.log_metric("test -" + "precision", seas_svm_unilabel_no_vacc_test_precision)
mlflow.log_metric("train -" + "f1", seas_svm_unilabel_no_vacc_train_f1)
mlflow.log_metric("test -" + "f1", seas_svm_unilabel_no_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

## Single Label Modelling, output seasonal vaccine -> H1N1 Flu Vaccine is in features

The y_seas_vacc remains the same from the previous model; the X feature and cat_features (for the preprocessor) need to be adjusted:

In [ ]:
cat_features_h1n1_vacc = cat_features.copy()

In [ ]:
cat_features_h1n1_vacc.remove('seasonal_vaccine')

In [ ]:
#NB: dropping the 'h1n1_vaccine' and 'seasonal_vaccine' columns
X_h1n1_vacc = df.drop(columns=['seasonal_vaccine'])

Performing test-train split (the same data can be used for each model in multilabelling):

In [ ]:
X_h1n1_vacc_train, X_h1n1_vacc_test, y_seas_vacc_train, y_seas_vacc_test = train_test_split(X_h1n1_vacc, y_seas_vacc, stratify = y_seas_vacc, test_size=0.2, random_state=RSEED)

In [ ]:
print('X_h1n1_vacc_train shape:', X_h1n1_vacc_train.shape)
print('X_h1n1_vacc_test shape:', X_h1n1_vacc_test.shape)
print('y_seas_vacc_train:', y_seas_vacc_train.shape)
print('y_seas_vacc_test:', y_seas_vacc_test.shape)

The preprocessor is adjusted:

In [ ]:
preprocessor_h1n1_vacc = ColumnTransformer([
    ('cat', cat_pipeline, cat_features_h1n1_vacc)
])

The pipeline is adjusted:

In [ ]:
# for logreg
logreg_h1n1_vacc_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor_h1n1_vacc),
    ("estimators", logreg),
])

# for KNN
knn_h1n1_vacc_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor_h1n1_vacc),
    ("estimators", knn),
])

# for Random Forest

rand_forst_h1n1_vacc_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor_h1n1_vacc),
    ("estimators", rand_forst),
])

#for SVM
svm_h1n1_vacc_unilabel_pipeline = Pipeline([
    ("preprocessor", preprocessor_h1n1_vacc),
    ("estimators", svm),
])

Training the models:

In [ ]:
# for logreg
logreg_unilabel_h1n1_vacc = logreg_h1n1_vacc_unilabel_pipeline.fit(X_h1n1_vacc_train, y_seas_vacc_train)

# for KNN
knn_unilabel_h1n1_vacc = knn_h1n1_vacc_unilabel_pipeline.fit(X_h1n1_vacc_train, y_seas_vacc_train)

# for Random Forest
rand_forst_unilabel_h1n1_vacc = rand_forst_h1n1_vacc_unilabel_pipeline.fit(X_h1n1_vacc_train, y_seas_vacc_train)

#for SVM
svm_unilabel_h1n1_vacc = svm_h1n1_vacc_unilabel_pipeline.fit(X_h1n1_vacc_train, y_seas_vacc_train)

In [ ]:
# Figure out later what this does and if we want to use it


#y_train_predicted = cross_val_predict(full_pipeline, X_train, y_train, cv=5)

Making predictions based on train and test data:

In [ ]:
# for logreg
logreg_unilabel_h1n1_vacc_trainpreds = logreg_unilabel_h1n1_vacc.predict(X_h1n1_vacc_train)
logreg_unilabel_h1n1_vacc_testpreds = logreg_unilabel_h1n1_vacc.predict(X_h1n1_vacc_test)

# for KNN
knn_unilabel_h1n1_vacc_trainpreds = knn_unilabel_h1n1_vacc.predict(X_h1n1_vacc_train)
knn_unilabel_h1n1_vacc_testpreds = knn_unilabel_h1n1_vacc.predict(X_h1n1_vacc_test)

# for Random Forest
rand_forst_unilabel_h1n1_vacc_trainpreds = rand_forst_unilabel_h1n1_vacc.predict(X_h1n1_vacc_train)
rand_forst_unilabel_h1n1_vacc_testpreds = rand_forst_unilabel_h1n1_vacc.predict(X_h1n1_vacc_test)

# for SVM
svm_unilabel_h1n1_vacc_trainpreds = svm_unilabel_h1n1_vacc.predict(X_h1n1_vacc_train)
svm_unilabel_h1n1_vacc_testpreds = svm_unilabel_h1n1_vacc.predict(X_h1n1_vacc_test)

### Model evaluation

#### Train data

In [ ]:
# Logreg--Train data evaluation Metrics
seas_logreg_unilabel_h1n1_vacc_train_acc = accuracy_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)
seas_logreg_unilabel_h1n1_vacc_train_recall = recall_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)
seas_logreg_unilabel_h1n1_vacc_train_precision = precision_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)
seas_logreg_unilabel_h1n1_vacc_train_f1 = f1_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)
seas_logreg_unilabel_h1n1_vacc_train_roc = roc_auc_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_train, logreg_unilabel_h1n1_vacc_trainpreds)))

In [ ]:
# KNN--Train data evaluation Metrics
seas_knn_unilabel_h1n1_vacc_train_acc = accuracy_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)
seas_knn_unilabel_h1n1_vacc_train_recall = recall_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)
seas_knn_unilabel_h1n1_vacc_train_precision = precision_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)
seas_knn_unilabel_h1n1_vacc_train_f1 = f1_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)
seas_knn_unilabel_h1n1_vacc_train_roc = roc_auc_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_train, knn_unilabel_h1n1_vacc_trainpreds)))

In [ ]:
# Random Forest--Train data evaluation Metrics
#check if these metrics are changing compared to previous results
seas_rand_forst_unilabel_h1n1_vacc_train_acc = accuracy_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)
seas_rand_forst_unilabel_h1n1_vacc_train_recall = recall_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)
seas_rand_forst_unilabel_h1n1_vacc_train_precision = precision_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)
seas_rand_forst_unilabel_h1n1_vacc_train_f1 = f1_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)
seas_rand_forst_unilabel_h1n1_vacc_train_roc = roc_auc_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_train, rand_forst_unilabel_h1n1_vacc_trainpreds)))

In [ ]:
# SVM--Train data evaluation Metrics
seas_svm_unilabel_h1n1_vacc_train_acc = accuracy_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)
seas_svm_unilabel_h1n1_vacc_train_recall = recall_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)
seas_svm_unilabel_h1n1_vacc_train_precision = precision_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)
seas_svm_unilabel_h1n1_vacc_train_f1 = f1_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)
seas_svm_unilabel_h1n1_vacc_train_roc = roc_auc_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_train, svm_unilabel_h1n1_vacc_trainpreds)))

#### Test data

In [ ]:
# Logreg--Test data evaluation Metrics
seas_logreg_unilabel_h1n1_vacc_test_acc = accuracy_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)
seas_logreg_unilabel_h1n1_vacc_test_recall = recall_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)
seas_logreg_unilabel_h1n1_vacc_test_precision = precision_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)
seas_logreg_unilabel_h1n1_vacc_test_f1 = f1_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)
seas_logreg_unilabel_h1n1_vacc_test_roc = roc_auc_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_test, logreg_unilabel_h1n1_vacc_testpreds)))

In [ ]:
# KNN--Test data evaluation Metrics
seas_knn_unilabel_h1n1_vacc_test_acc = accuracy_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)
seas_knn_unilabel_h1n1_vacc_test_recall = recall_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)
seas_knn_unilabel_h1n1_vacc_test_precision = precision_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)
seas_knn_unilabel_h1n1_vacc_test_f1 = f1_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)
seas_knn_unilabel_h1n1_vacc_test_roc = roc_auc_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_test, knn_unilabel_h1n1_vacc_testpreds)))

In [ ]:
# Random Forest--Test data evaluation Metrics
#check if these metrics are changing compared to previous results
seas_rand_forst_unilabel_h1n1_vacc_test_acc = accuracy_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)
seas_rand_forst_unilabel_h1n1_vacc_test_recall = recall_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)
seas_rand_forst_unilabel_h1n1_vacc_test_precision = precision_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)
seas_rand_forst_unilabel_h1n1_vacc_test_f1 = f1_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)
seas_rand_forst_unilabel_h1n1_vacc_test_roc = roc_auc_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_test, rand_forst_unilabel_h1n1_vacc_testpreds)))

In [ ]:
# SVM--Test data evaluation Metrics
seas_svm_unilabel_h1n1_vacc_test_acc = accuracy_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)
seas_svm_unilabel_h1n1_vacc_test_recall = recall_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)
seas_svm_unilabel_h1n1_vacc_test_precision = precision_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)
seas_svm_unilabel_h1n1_vacc_test_f1 = f1_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)
seas_svm_unilabel_h1n1_vacc_test_roc = roc_auc_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)

#print("Accuracy: {:.2f}".format(accuracy_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)))
#print("Recall: {:.2f}".format(recall_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)))
#print("Precision: {:.2f}".format(precision_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)))
#print("F1: {:.2f}".format(f1_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)))
#print("ROC: {:.2f}".format(roc_auc_score(y_seas_vacc_test, svm_unilabel_h1n1_vacc_testpreds)))

## Tracking the model with MLFlow

### H1N1 vaccine output

#### Logistic regression

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_seasonal) #this needs to be adjusted for each experiment
name = 'logreg_unilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "H1N1") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seas_logreg_unilabel_h1n1_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", seas_logreg_unilabel_h1n1_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", seas_logreg_unilabel_h1n1_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", seas_logreg_unilabel_h1n1_vacc_test_acc)
mlflow.log_metric("train -" + "recall", seas_logreg_unilabel_h1n1_vacc_train_recall)
mlflow.log_metric("test -" + "recall", seas_logreg_unilabel_h1n1_vacc_test_recall)
mlflow.log_metric("train -" + "precision", seas_logreg_unilabel_h1n1_vacc_train_precision)
mlflow.log_metric("test -" + "precision", seas_logreg_unilabel_h1n1_vacc_test_precision)
mlflow.log_metric("train -" + "f1", seas_logreg_unilabel_h1n1_vacc_train_f1)
mlflow.log_metric("test -" + "f1", seas_logreg_unilabel_h1n1_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### KNN

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_seasonal)
name = 'knn_unilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "H1N1") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seas_knn_unilabel_h1n1_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", seas_knn_unilabel_h1n1_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", seas_knn_unilabel_h1n1_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", seas_knn_unilabel_h1n1_vacc_test_acc)
mlflow.log_metric("train -" + "recall", seas_knn_unilabel_h1n1_vacc_train_recall)
mlflow.log_metric("test -" + "recall", seas_knn_unilabel_h1n1_vacc_test_recall)
mlflow.log_metric("train -" + "precision", seas_knn_unilabel_h1n1_vacc_train_precision)
mlflow.log_metric("test -" + "precision", seas_knn_unilabel_h1n1_vacc_test_precision)
mlflow.log_metric("train -" + "f1", seas_knn_unilabel_h1n1_vacc_train_f1)
mlflow.log_metric("test -" + "f1", seas_knn_unilabel_h1n1_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### Random forest

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_seasonal)
name = 'rand_forst_unilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "H1N1") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seas_rand_forst_unilabel_h1n1_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", seas_rand_forst_unilabel_h1n1_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", seas_rand_forst_unilabel_h1n1_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", seas_rand_forst_unilabel_h1n1_vacc_test_acc)
mlflow.log_metric("train -" + "recall", seas_rand_forst_unilabel_h1n1_vacc_train_recall)
mlflow.log_metric("test -" + "recall", seas_rand_forst_unilabel_h1n1_vacc_test_recall)
mlflow.log_metric("train -" + "precision", seas_rand_forst_unilabel_h1n1_vacc_train_precision)
mlflow.log_metric("test -" + "precision", seas_rand_forst_unilabel_h1n1_vacc_test_precision)
mlflow.log_metric("train -" + "f1", seas_rand_forst_unilabel_h1n1_vacc_train_f1)
mlflow.log_metric("test -" + "f1", seas_rand_forst_unilabel_h1n1_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)

#### SVM

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME_seasonal)
name = 'svm_unilabel_seasonal' #specify the run name here; name it with model used
mlflow.start_run(run_name = name) 
run = mlflow.active_run()

In [ ]:
print("Active run_id: {}".format(run.info.run_id))

In [ ]:
#parameters are to keep track of everything for re-running the experiment (e.g. what features are being engineered, hyperparameters of model, etc)
#to be adjusted as needed
params = {
    "Data cleaning": "Full dataset",
    "Data balancing": "None",
    "Hyperparameters": "None"
  }

In [ ]:
mlflow.log_params(params)
#tags = data used
mlflow.set_tag("Vaccines in features", "H1N1") #set tags for more details of what we've done
mlflow.log_metric("train -" + "ROC", seas_svm_unilabel_h1n1_vacc_train_roc)
mlflow.log_metric("test -" + "ROC", seas_svm_unilabel_h1n1_vacc_test_roc)
mlflow.log_metric("train -" + "accuracy", seas_svm_unilabel_h1n1_vacc_train_acc)
mlflow.log_metric("test -" + "accuracy", seas_svm_unilabel_h1n1_vacc_test_acc)
mlflow.log_metric("train -" + "recall", seas_svm_unilabel_h1n1_vacc_train_recall)
mlflow.log_metric("test -" + "recall", seas_svm_unilabel_h1n1_vacc_test_recall)
mlflow.log_metric("train -" + "precision", seas_svm_unilabel_h1n1_vacc_train_precision)
mlflow.log_metric("test -" + "precision", seas_svm_unilabel_h1n1_vacc_test_precision)
mlflow.log_metric("train -" + "f1", seas_svm_unilabel_h1n1_vacc_train_f1)
mlflow.log_metric("test -" + "f1", seas_svm_unilabel_h1n1_vacc_test_f1)
#mlflow.log_artifact("../models")
#mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

In [ ]:
#mlflow.get_run(run_id=run.info.run_id)